# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [4]:
text_input = driver.find_element_by_name('OperSearch')

In [5]:
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)

In [6]:
text_input.send_keys('dirt')

In [7]:
abandoned_checkbox = driver.find_element_by_name('Abandoned')
abandoned_checkbox.click()

In [8]:
button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

In [37]:
# [6] is column headers
# [7] is first bit of pay dirt
# columns: ID State Operator Mine Name Type CM* Status Commodity More Info

dirt_operators = driver.find_elements_by_tag_name('tr')
rows = []

for operator in dirt_operators[7:-1]:
    row = {}
    
    cells = operator.find_elements_by_tag_name('td')
    
    ID = cells[0].text
    row['ID'] = ID
    
    State = cells[1].text
    row['State'] = State
    
    Operator = cells[2].text
    row['Operator'] = Operator
    
    Mine_Name = cells[3].text
    row['Mine_Name'] = Mine_Name
    
    Type = cells[4].text
    row['Type'] = Type
    
    CM = cells[5].text
    row['Coal_or_Metal'] = CM
    
    Status = cells[6].text
    row['Status'] = Status
    
    Commodity = cells[7].text
    row['Commodity'] = Commodity
    
    rows.append(row)
rows

[{'ID': '3503598',
  'State': 'OR ',
  'Operator': 'Newberg Rock & Dirt  ',
  'Mine_Name': 'Newberg Rock & Dirt',
  'Type': 'Surface',
  'Coal_or_Metal': 'M ',
  'Status': 'Active ',
  'Commodity': 'Crushed, Broken Stone NEC '},
 {'ID': '0502030',
  'State': 'CO ',
  'Operator': 'Allied Dirt Moving Company  ',
  'Mine_Name': 'Allied Dirt Moving Co Pit & Plant',
  'Type': 'Surface',
  'Coal_or_Metal': 'M ',
  'Status': 'Abandoned ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '4801789',
  'State': 'ND ',
  'Operator': 'AM Dirtworks & Aggregate Sales  ',
  'Mine_Name': 'AM Dirtworks & Aggregate Sales',
  'Type': 'Surface',
  'Coal_or_Metal': 'M ',
  'Status': 'Abandoned ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '4201449',
  'State': 'UT ',
  'Operator': 'Atlas-Dirty Devil Mining  ',
  'Mine_Name': 'Unit Train Loading Facility',
  'Type': 'Facility',
  'Coal_or_Metal': 'C ',
  'Status': 'Abandoned ',
  'Commodity': 'Coal (Bituminous) '},
 {'ID': '4201450',
 

In [38]:
import pandas as pd
df = pd.DataFrame(rows)
df.head()

,Coal_or_Metal,Commodity,ID,Mine_Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,0502030,Allied Dirt Moving Co Pit & Plant,Allied Dirt Moving Company,CO,Abandoned,Surface
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Abandoned,Surface
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Atlas-Dirty Devil Mining,UT,Abandoned,Facility
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Atlas-Dirty Devil Mining,UT,Abandoned,Surface


In [43]:
df.to_csv("dirt-operators.csv", index=False)

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [48]:
dirt_operators = driver.find_elements_by_tag_name('tr')
for operator in dirt_operators[7:-1]:
    cells = operator.find_elements_by_tag_name('td')
    print(cells[2].text)

Newberg Rock & Dirt  
Allied Dirt Moving Company  
AM Dirtworks & Aggregate Sales  
Atlas-Dirty Devil Mining  
Atlas-Dirty Devil Mining  
Babe's Dirt Work  
Bar-Lin Dirt Company  
Barber'S Dirt Pit  
Bender Sand & Dirt  
BERT'S DIRT  
Big D Dirt Service Inc  
Big Red Dirt Farm LLC  
Big River Dirt Pit  
Bob Harris Dirt Contracting  
Bohannon Sand & Dirt  
Bratcher'S Sand & Dirt  
Brewer Dirt Works  
Buck'S Dirt Pit  
C & G Dirt Hauling  
C N C Dirt Movers, Inc.  
Cambridge Dirt Sand and Gravel LLC  
Central Iowa Dirt & Demo LLC  
Crowes Trucking & Dirt Pit Services  
D & H Dirt  
Diez Dirt & Sand Hauling Inc  
Dirt Cheap  
Dirt Company  
Dirt Company  
Dirt Company  
Dirt Con  
Dirt Diggers Inc  
Dirt Doctor Inc  
Dirt Inc  
Dirt Pit  
Dirt Work Specialists LLC  
Dirt Works  
Dirtco Inc  
Dirtman Trucking  
DIRTWORKS, INC.  
Dirtworks, Inc.  
Dirty Coal  
Dorchester Dirt Company Inc  
Douglas Dirt Sand & Gravel Company  
Ell Dirt Works LLC.  
Floyd Smith Dirt Pit  
Gary Kelm Dirt Servi

### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [49]:
for operator in dirt_operators[7:-1]:
    cells = operator.find_elements_by_tag_name('td')
    print(cells[0].text)

3503598
0502030
4801789
4201449
4201450
1002257
1601167
4103265
1401575
1700776
1601251
0301963
1601082
3401751
1600916
3401211
0301267
1600956
2200033
0504953
3401929
1302445
1601106
3400915
1600983
4503200
3401266
3401468
5001797
4608254
1510279
2103723
0100776
4104016
2103914
4104757
0301729
0404851
2200734
5002028
1513393
3800602
3101630
3200860
3401762
2103517
2402626
2103181
1601124
1601150
4703427
0801306
2501216
3200965
2901371
2901544
2901709
4102355
4102420
4102869
4102951
4102958
4104876
3003502
4103258
3901432
2103556
1601250
1600908
1600953
4104185
2901536
3609624
3800709
3609931
1601257
0801275
1601379
1601380
1601381
1601134
1601165
3901042
1601194
4104054
4801674
2402474
1600920
4102955
4103107
1512530
1515619
1518318
4405366
4407196
1519685
1519799
4407379
4407003
2602570
2402503
4407296
1519273
4407270
4102682
0801259
0203332
0302015
2901986
1601127
4105017
1600986
4103324
4202013
0801417
0801371
2402115
4300748
4300768
4300776
0103209
1601159
2302283
4102586
4104475


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [ ]:
# did this above

### Save that to a CSV named `dirt-operators.csv`

In [ ]:
# did this above

In [ ]:
df = pd.read_csv("Users\matthewrehbein\LEDE\Foundations\github_foundations\Found_HW7\dirt_operators.csv")
df.head()

df = pd.read_csv("Users/matthewrehbein/LEDE/Foundations/github_foundations/Found_HW7/dirt_operators.csv")
df.head()

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [57]:
df = pd.read_csv("dirt-operators.csv")
df.head()

,Coal_or_Metal,Commodity,ID,Mine_Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,502030,Allied Dirt Moving Co Pit & Plant,Allied Dirt Moving Company,CO,Abandoned,Surface
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Abandoned,Surface
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Atlas-Dirty Devil Mining,UT,Abandoned,Facility
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Atlas-Dirty Devil Mining,UT,Abandoned,Surface
